In [ ]:
!pip install -U d2l
!python -m pip install paddlepaddle -i https://mirror.baidu.com/pypi/simple

In [ ]:
import paddle
import matplotlib.pyplot as plt
import numpy as np

# softmax回归的从0开始实现

就像我们从零开始实现线性回归一样，我们认为softmax回归也是重要的基础，因此你应该知道实现softmax的细节。我们使用刚刚在 3.5节中引入的Fashion-MNIST数据集，并设置数据迭代器的批量大小为256。

#### torch版

In [ ]:
import torch
from IPython import display
from d2l import torch as d2l

batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

#### paddle版

In [ ]:
def paddle_load_data_fashion_mnist(batch_size, resize=None):
    """下载Fashion-MNIST数据集，然后将其加载到内存中。"""
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0, transforms.Resize(resize))
    trans = transforms.Compose(trans)
    paddle_transform = T.Compose([T.Normalize(mean=[127.5],std=[127.5],data_format='CHW')])
    paddle_train = paddle.vision.datasets.FashionMNIST(mode='train', transform=paddle_transform)
    paddle_test = paddle.vision.datasets.FashionMNIST(mode='test', transform=paddle_transform)
    return (paddle.io.DataLoader(paddle_train, batch_size, shuffle=True,
                            num_workers=paddle_get_dataloader_workers()),
            paddle.io.DataLoader(paddle_test, batch_size, shuffle=False,
                            num_workers=paddle_get_dataloader_workers()))

In [ ]:
batch_size = 256
paddle_train_iter, paddle_test_iter = d2l.load_data_fashion_mnist(batch_size)

## 3.6.1 初始化模型参数

和之前线性回归的例子一样，这里的每个样本都将用固定长度的向量表示。原始数据集中的每个样本都是 28×28 的图像。在本节中，我们将展平每个图像，把它们看作长度为784的向量。在后面的章节中，将讨论能够利用图像空间结构的更为复杂的策略，但现在我们暂时只把每个像素位置看作一个特征。

回想一下，在softmax回归中，我们的输出与类别一样多。因为我们的数据集有10个类别，所以网络输出维度为10。因此，权重将构成一个 784×10 的矩阵，偏置将构成一个 1×10 的行向量。与线性回归一样，我们将使用正态分布初始化我们的权重W，偏置初始化为0。

#### torch版



In [ ]:
num_inputs = 784
num_outputs = 10

W = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True)
b = torch.zeros(num_outputs, requires_grad=True)

#### paddle版

In [ ]:
num_inputs = 784
num_outputs = 10

paddle_W = paddle.normal(0, 0.01, shape=(num_inputs, num_outputs))
paddle_b = paddle.zeros(shape=(num_outputs,1))

## 3.6.2 定义softmax操作

在实现softmax回归模型之前，让我们简要地回顾一下sum运算符如何沿着张量中的特定维度工作，如 2.3.6节和 2.3.6.1节所述。给定一个矩阵X，我们可以对所有元素求和（默认情况下），也可以只求同一个轴上的元素，即同一列（轴0）或同一行（轴1）。如果X是一个形状为(2, 3)的张量，我们对列进行求和，则结果将是一个具有形状(3,)的向量。当调用sum运算符时，我们可以指定保持在原始张量的轴数，而不折叠求和的维度。这将产生一个具有形状(1, 3)的二维张量。

#### torch版

In [ ]:
X = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
X.sum(0, keepdim=True), X.sum(1, keepdim=True)

#### paddle版

In [ ]:
paddle_X = paddle.to_tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
paddle_X.sum(0, keepdim=True), paddle_X.sum(1, keepdim=True)

我们现在已经准备好实现softmax操作了。回想一下，softmax由三个步骤组成:
 

*   对每个项求幂（使用exp）；
*   对每一行求和（小批量中每个样本是一行），得到每个样本的归一化常数；
*   将每一行除以其归一化常数，确保结果的和为1。

在查看代码之前，让我们回顾一下这个表达式

$$softmax(X)_{ij}=\frac{exp(X_{ij})}{\sum_k exp(X_{ik})}$$
 
分母或归一化常数，有时也称为配分函数（其对数称为对数-配分函数）。该名称的起源来自统计物理学中一个模拟粒子群分布的方程。

In [ ]:
def softmax(X):
    X_exp = torch.exp(X)
    partition = X_exp.sum(1, keepdim=True)
    return X_exp / partition  # 这里应用了广播机制

正如你所看到的，对于任何随机输入，我们将每个元素变成一个非负数。此外，依据概率原理，每行总和为1。


In [ ]:
X = torch.normal(0, 1, (2, 5))
X_prob = softmax(X)
X_prob, X_prob.sum(1)

#### paddle版

In [ ]:
def paddle_softmax(X):
    paddle_X_exp = paddle.exp(X)
    paddle_partition = paddle_X_exp.sum(1, keepdim=True)
    return paddle_X_exp / paddle_partition  # 这里应用了广播机制

In [ ]:
paddle_X = paddle.normal(0, 1, (2, 5))
paddle_X_prob = paddle_softmax(paddle_X)
paddle_X_prob, paddle_X_prob.sum(1)

## 3.6.3 定义模型

现在我们已经定义了softmax操作，我们可以实现softmax回归模型。下面的代码定义了输入如何通过网络映射到输出。注意，在将数据传递到我们的模型之前，我们使用reshape函数将每张原始图像展平为向量。

#### torch版

In [ ]:
def net(X):
    return softmax(torch.matmul(X.reshape((-1, W.shape[0])), W) + b)

#### paddle版

In [ ]:
def paddle_net(X):
    return paddle_softmax(paddle.matmul(X.reshape((-1, paddle_W.shape[0])), paddle_W) + paddle_b)

## 3.6.4 定义损失函数

接下来，我们需要实现 3.4节中引入的交叉熵损失函数。这可能是深度学习中最常见的损失函数，因为目前分类问题的数量远远超过回归问题。

回顾一下，交叉熵采用真实标签的预测概率的负对数似然。我们不需要使用Python的for循环迭代预测（这往往是低效的）。我们可以通过一个运算符选择所有元素。 下面，我们创建一个数据y_hat，其中包含2个样本在3个类别的预测概率，它们对应的标签y。 有了y，我们知道在第一个样本中，第一类是正确的预测，而在第二个样本中，第三类是正确的预测。 然后使用y作为y_hat中概率的索引，我们选择第一个样本中第一个类的概率和第二个样本中第三个类的概率。

#### torch版

In [ ]:
y = torch.tensor([0, 2])
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y_hat[[0, 1], y]

In [ ]:
def cross_entropy(y_hat, y):
    return - torch.log(y_hat[range(len(y_hat)), y])

cross_entropy(y_hat, y)

#### paddle版

In [ ]:
paddle_y = paddle.to_tensor([0, 2])
paddle_y_hat = paddle.to_tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
paddle_y_hat[[0, 1], paddle_y]

In [ ]:
def paddle_cross_entropy(y_hat, y):
    return - paddle.log(y_hat[range(len(y_hat)), y])

cross_entropy(paddle_y_hat, paddle_y)

## 3.6.5 分类准确率

给定预测概率分布y_hat，当我们必须输出硬预测（hard prediction）时，我们通常选择预测概率最高的类。许多应用都要求我们做出选择。如Gmail必须将电子邮件分为“Primary（主要）”、“Social（社交）”、“Updates（更新）”或“Forums（论坛）”。它可能在内部估计概率，但最终它必须在类中选择一个。

当预测与标签分类y一致时，它们是正确的。分类准确率即正确预测数量与总预测数量之比。虽然直接优化准确率可能很困难（因为准确率的计算不可导），但准确率通常是我们最关心的性能衡量标准，我们在训练分类器时几乎总是会报告它。

为了计算准确率，我们执行以下操作。首先，如果y_hat是矩阵，那么假定第二个维度存储每个类的预测分数。我们使用argmax获得每行中最大元素的索引来获得预测类别。然后我们将预测类别与真实y元素进行比较。由于等式运算符“==”对数据类型很敏感，因此我们将y_hat的数据类型转换为与y的数据类型一致。结果是一个包含0（错）和1（对）的张量。进行求和会得到正确预测的数量。

#### torch版

In [ ]:
def accuracy(y_hat, y): 
    """计算预测正确的数量。"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

我们将继续使用之前定义的变量y_hat和y分别作为预测的概率分布和标签。我们可以看到，第一个样本的预测类别是2（该行的最大元素为0.6，索引为2），这与实际标签0不一致。第二个样本的预测类别是2（该行的最大元素为0.5，索引为2），这与实际标签2一致。因此，这两个样本的分类准确率率为0.5。

In [ ]:
accuracy(y_hat, y) / len(y)

同样，对于任意数据迭代器data_iter可访问的数据集，我们可以评估在任意模型net的准确率。


In [ ]:
def evaluate_accuracy(net, data_iter):
    """计算在指定数据集上模型的精度。"""
    if isinstance(net, torch.nn.Module):
        net.eval()  # 将模型设置为评估模式
    metric = Accumulator(2)  # 正确预测数、预测总数
    for X, y in data_iter:
        metric.add(accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]

这里Accumulator是一个实用程序类，用于对多个变量进行累加。 在上面的evaluate_accuracy函数中，我们在Accumulator实例中创建了2个变量，用于分别存储正确预测的数量和预测的总数量。当我们遍历数据集时，两者都将随着时间的推移而累加。

In [ ]:
class Accumulator: 
    """在`n`个变量上累加。"""
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

由于我们使用随机权重初始化net模型，因此该模型的准确率应接近于随机猜测。例如在有10个类别情况下的准确率为0.1。

In [ ]:
evaluate_accuracy(net, test_iter)

#### paddle版

In [ ]:
def paddle_accuracy(y_hat, y):
    """计算预测正确的数量。"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

In [ ]:
accuracy(paddle_y_hat, paddle_y) / len(paddle_y)

In [ ]:
def paddle_evaluate_accuracy(net, data_iter):
    """计算在指定数据集上模型的精度。"""
    if isinstance(net, paddle.nn.Module):
        net.eval()  # 将模型设置为评估模式
    metric = Accumulator(2)  # 正确预测数、预测总数
    for X, y in data_iter:
        metric.add(accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]

In [ ]:
paddle_evaluate_accuracy(paddle_net, paddle_test_iter)

## 3.6.6 训练

#### torch版

In [ ]:
def train_epoch_ch3(net, train_iter, loss, updater):
    """训练模型一个迭代周期（定义见第3章）。"""
    # 将模型设置为训练模式
    if isinstance(net, torch.nn.Module):
        net.train()
    # 训练损失总和、训练准确度总和、样本数
    metric = Accumulator(3)
    for X, y in train_iter:
        # 计算梯度并更新参数
        y_hat = net(X)
        l = loss(y_hat, y)
        if isinstance(updater, torch.optim.Optimizer):
            # 使用PyTorch内置的优化器和损失函数
            updater.zero_grad()
            l.backward()
            updater.step()
            metric.add(float(l) * len(y), accuracy(y_hat, y),
                       y.size().numel())
        else:
            # 使用定制的优化器和损失函数
            l.sum().backward()
            updater(X.shape[0])
            metric.add(float(l.sum()), accuracy(y_hat, y), y.numel())
    # 返回训练损失和训练准确率
    return metric[0] / metric[2], metric[1] / metric[2]

在展示训练函数的实现之前，我们定义一个在动画中绘制数据的实用程序类。它能够简化本书其余部分的代码。



In [ ]:
class Animator:
    """在动画中绘制数据。"""
    def __init__(self, xlabel=None, ylabel=None, legend=None, xlim=None,
                 ylim=None, xscale='linear', yscale='linear',
                 fmts=('-', 'm--', 'g-.', 'r:'), nrows=1, ncols=1,
                 figsize=(3.5, 2.5)):
        # 增量地绘制多条线
        if legend is None:
            legend = []
        d2l.use_svg_display()
        self.fig, self.axes = d2l.plt.subplots(nrows, ncols, figsize=figsize)
        if nrows * ncols == 1:
            self.axes = [self.axes, ]
        # 使用lambda函数捕获参数
        self.config_axes = lambda: d2l.set_axes(
            self.axes[0], xlabel, ylabel, xlim, ylim, xscale, yscale, legend)
        self.X, self.Y, self.fmts = None, None, fmts

    def add(self, x, y):
        # 向图表中添加多个数据点
        if not hasattr(y, "__len__"):
            y = [y]
        n = len(y)
        if not hasattr(x, "__len__"):
            x = [x] * n
        if not self.X:
            self.X = [[] for _ in range(n)]
        if not self.Y:
            self.Y = [[] for _ in range(n)]
        for i, (a, b) in enumerate(zip(x, y)):
            if a is not None and b is not None:
                self.X[i].append(a)
                self.Y[i].append(b)
        self.axes[0].cla()
        for x, y, fmt in zip(self.X, self.Y, self.fmts):
            self.axes[0].plot(x, y, fmt)
        self.config_axes()
        display.display(self.fig)
        display.clear_output(wait=True)


接下来我们实现一个训练函数，它会在train_iter访问到的训练数据集上训练一个模型net。该训练函数将会运行多个迭代周期（由num_epochs指定）。在每个迭代周期结束时，利用test_iter访问到的测试数据集对模型进行评估。我们将利用Animator类来可视化训练进度。

In [ ]:
def train_ch3(net, train_iter, test_iter, loss, num_epochs, updater): 
    """训练模型（定义见第3章）。"""
    animator = Animator(xlabel='epoch', xlim=[1, num_epochs], ylim=[0.3, 0.9],
                        legend=['train loss', 'train acc', 'test acc'])
    for epoch in range(num_epochs):
        train_metrics = train_epoch_ch3(net, train_iter, loss, updater)
        test_acc = evaluate_accuracy(net, test_iter)
        animator.add(epoch + 1, train_metrics + (test_acc,))
    train_loss, train_acc = train_metrics
    assert train_loss < 0.5, train_loss
    assert train_acc <= 1 and train_acc > 0.7, train_acc
    assert test_acc <= 1 and test_acc > 0.7, test_acc

作为一个从零开始的实现，我们使用 3.2节中定义的小批量随机梯度下降来优化模型的损失函数，设置学习率为0.1。

In [ ]:
lr = 0.1

def updater(batch_size):
    return d2l.sgd([W, b], lr, batch_size)

现在，我们训练模型10个迭代周期。请注意，迭代周期（num_epochs）和学习率（lr）都是可调节的超参数。通过更改它们的值，我们可以提高模型的分类准确率。

In [ ]:
num_epochs = 10
train_ch3(net, train_iter, test_iter, cross_entropy, num_epochs, updater)

## 3.6.7预测

现在训练已经完成，我们的模型已经准备好对图像进行分类预测。给定一系列图像，我们将比较它们的实际标签（文本输出的第一行）和模型预测（文本输出的第二行）。

In [ ]:
def predict_ch3(net, test_iter, n=6):
    """预测标签（定义见第3章）。"""
    for X, y in test_iter:
        break
    trues = d2l.get_fashion_mnist_labels(y)
    preds = d2l.get_fashion_mnist_labels(net(X).argmax(axis=1))
    titles = [true +'\n' + pred for true, pred in zip(trues, preds)]
    d2l.show_images(
        X[0:n].reshape((n, 28, 28)), 1, n, titles=titles[0:n])

predict_ch3(net, test_iter)